In [1]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

<ipython-input-1-32e0d8fb07c9>:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1788.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])


In [2]:
img_t = torch.randn(3, 5, 5)  #創建一個3軸張量
weights = torch.tensor([0.2126, 0.7152, 0.0722])  

In [3]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [4]:
img_t = torch.randn(3, 5, 5)  #創建一個3軸張量
batch_t = torch.randn(2, 3, 5, 5)   #創建一個4軸張量
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [5]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [6]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [7]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [8]:
#將第0軸命名為channels
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [9]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [10]:
weights_aligned = weights_named.align_as(img_named)   #為1軸張量，只有channels
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [11]:
gray_named = (img_named * weights_aligned).sum('channels')    #sum()接受軸名稱當參數
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [12]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [13]:
gray_plain = gray_named.rename(None)  #將張量變回未命名
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

In [14]:
double_points = torch.ones(10, 2, dtype=torch.double)   #64位元雙精度浮點數
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)    #16位元整數

In [15]:
short_points.dtype

torch.int16

In [16]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()
print(double_points)
print(short_points)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)
tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]], dtype=torch.int16)


In [17]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)
print(double_points.dtype)
print(short_points.dtype)

torch.float64
torch.int16


In [18]:
points_64 = torch.zeros(5, dtype=torch.double)    #建立內含5個元素的64位元浮點數張量，皆為0
points_short = points_64.to(torch.short)      #設定points_short為16位元整數張量
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [19]:
a = torch.ones(3, 2)  #創建一個3x2的2軸張量
a_t = torch.transpose(a, 0, 1)  #將a張量的第0軸及第1軸進行轉置
a.shape, a_t.shape  #印出a張量及a_t張量的shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [20]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)   #將a張量的第0軸及第1軸進行轉置
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [21]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])   #創建一個3x2的2軸張量，存放三個座標點
points.storage()    #查看points的storage

<ipython-input-21-97c45f812328>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()    #查看points的storage
/usr/local/lib/python3.9/dist-packages/IPython/lib/pretty.py:700: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  output = repr(obj)
/usr/local/lib/python3.9/dist-packages/torch/storage.py:645: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access Unty

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [22]:
points_storage = points.storage()   #points_storage為points張量的storage
points_storage[0]   #取得points_storage中第0個元素

<ipython-input-22-420501d423d3>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage = points.storage()   #points_storage為points張量的storage
<ipython-input-22-420501d423d3>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage[0]   #取得points_storage中第0個元素


4.0

In [23]:
points.storage()[0]   #取得points_storage中第0個元素(另一種方式)

<ipython-input-23-8a585a2225d9>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()[0]   #取得points_storage中第0個元素(另一種方式)


4.0

In [24]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 19.0    #將points_storage中的索引為0的值改為19.0
points

<ipython-input-24-7e290c2f6b66>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage = points.storage()
<ipython-input-24-7e290c2f6b66>:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points_storage[0] = 19.0    #將points_storage中的索引為0的值改為19.0


tensor([[19.,  1.],
        [ 5.,  3.],
        [ 2.,  1.]])

In [25]:
a = torch.ones(3, 2)
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [26]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [27]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

<ipython-input-27-8575ff47c9e0>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [28]:
second_point = points[1]
second_point.storage()

<ipython-input-28-8bb992cd67ec>:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  second_point.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [29]:
second_point.storage_offset()   #輸出second_point張量的偏移量

2

In [30]:
second_point.size()

torch.Size([2])

In [31]:
second_point.shape

torch.Size([2])

In [32]:
points.stride()

(2, 1)

In [33]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [34]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 65.0  #將second_point中索引為0的元素值改成65.0
points    #檢查張量是否有變化

tensor([[ 4.,  1.],
        [65.,  3.],
        [ 2.,  1.]])

In [35]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()    #利用clone()複製一份points的張量中索引為1的座標資料

second_point[0] = 65.0  #將second_point中索引為0的元素值改成65.0
points    #檢查張量是否有變化

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [36]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

points_t = points.t()   #對points轉置，存成points_t
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [37]:
id(points.storage()) == id(points_t.storage())

<ipython-input-37-e76ded9f0529>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  id(points.storage()) == id(points_t.storage())


False

In [38]:
some_t = torch.ones(3, 4, 5)  #創建一個shape為3x4x5的3軸張量
print(some_t.shape)

transpose_t = some_t.transpose(0, 2)   #第0軸及第2軸進行轉置
print(transpose_t.shape)

print(some_t.stride())
print(transpose_t.stride())

torch.Size([3, 4, 5])
torch.Size([5, 4, 3])
(20, 5, 1)
(1, 5, 20)


In [42]:
#檢查張量是否為連續性
print(points.is_contiguous())
print(points_t.is_contiguous())

True
False


In [51]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()   #對points張量進行轉置
print(points_t)
print(points_t.storage())
print(points_t.stride())

tensor([[4., 5., 2.],
        [1., 3., 1.]])
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
(1, 2)


<ipython-input-51-993737f18d2e>:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(points_t.storage())
/usr/local/lib/python3.9/dist-packages/torch/storage.py:636: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  f'device={self.device}) of size {len(self)}]')
/usr/local/lib/python3.9/dist-packages/torch/storage.py:637: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To 

In [54]:
points_t_cont = points_t.contiguous()    #轉換成連續張量
print(points_t_cont)
print(points_t_cont.stride())
print(points_t_cont.storage())

tensor([[4., 5., 2.],
        [1., 3., 1.]])
(3, 1)
 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]


<ipython-input-54-ffd9a4bfcfb6>:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(points_t_cont.storage())


In [58]:
points = torch.ones(3, 4)
points_np = points.numpy()    #將Pytorch張量轉換成Numpy陣列
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [59]:
torch.save(points, 'outputs.t')

In [60]:
with open('output.t', 'wb') as f:
  torch.save(points, f)

In [61]:
points = torch.load('output.t')

In [62]:
with open('output.t', 'rb') as f:
  points = torch.load(f)

In [63]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
import h5py
f = h5py.File('output.hdf5', 'w')   #創建一個hdf5檔，並開啟寫入模式
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [68]:
f = h5py.File('output.hdf5', 'r')   #讀入存有各個座標點資訊的HDF5檔
dset = f['coords']
last_points = dset[-2:]   #取得最後兩個座標點的資訊
print(last_points)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
